# Toy problem for debugging Dual Decomposition

$$
\begin{align*}
\max_{x_k,y} \quad & \sum_k f_k'y + \sum_k c_k'x_k \\
\mathrm{s.t.} \quad & A_k x_k + B_k y \leq d_k, \\
& y\in \{0,1\}.
\end{align*}
$$

$$
\begin{align*}
\max_{x_k,y_k} \quad & \sum_k f_k'y + \sum_k c_k'x_k \\
\mathrm{s.t.} \quad & A_k x_k + B_k y_k \leq d_k, \\
& \sum_k H_k y_k = 0, \\
& y_k\in \{0,1\}.
\end{align*}
$$

In [1]:
RELAX_LP = False

In [2]:
from __future__ import division
from cobra import Model
from dynamicme.optimize import Variable, Constraint
from six import iteritems
import numpy as np

### Example from
http://www.optirisk-systems.com/manuals/FortspManual.pdf

$$
\begin{align*}
\max_{x,y} \quad & \sum_k P_k \left( \sum_c p^s_c x^s_{c,k} - p^b_c x^b_{c,k}  - \sum_c c_c a_c \right)\\
\mathrm{s.t.} \quad & \sum_c a_c \leq A \\
& Y_{c,k}  a_c - x^s_{c,k} + x^b_{c,k} \geq R_c \\
& x^s_{\mathrm{beet},k} \leq Q(\mathrm{beet}) \\
& x^s_{\mathrm{beet},c} + s^x_k \leq Y_{\mathrm{beet},k} a_\mathrm{beet}
\end{align*}
$$

In [3]:
from dynamicme.decomposition import LagrangeMaster, LagrangeSubmodel

In [4]:
#------------------------------------------------------------
# Data
crops = ['wheat','corn','beet']
total_area = 500.  # Total acres
prob_dict = {'below':1./3, 'average':1./3, 'above':1./3}
# Yields (tons/acre) over scenarios
yield_dict = {}
yield_dict['below'] = {'wheat':2., 'corn':2.4, 'beet':16.}
yield_dict['average'] = {'wheat':2.5, 'corn':3., 'beet':20.}
yield_dict['above'] = {'wheat':3., 'corn':3.6, 'beet':24.}
# Planting costs
plantcost_dict={'wheat':150, 'corn':230, 'beet':260}   # $/ton
sellprice_dict = {'wheat':170, 'corn':150, 'beet':36}  # $/ton
excess_sell_price = 10    # beet sell price ($/ton) when quota exceeded
beet_quota = 6000   # tons
buyprice_dict = {'wheat':238, 'corn':210, 'beet':100}  # $/ton
req_dict = {'wheat':200., 'corn':240., 'beet':0.} # tons required to feed cattle
#------------------------------------------------------------
# Area is complicating (first-stage) variable

In [5]:
# Create subproblems
UB = 1e4
sub_dict = {}
for scen,yields in iteritems(yield_dict):
    # Each subproblem gets its own copy of first-stage variables
    area_dict = {c:Variable('area_%s'%c, lower_bound=0., upper_bound=1e6,
        objective_coefficient=prob_dict[scen]*plantcost_dict[c]) for c in crops}
    for y in area_dict.values():
        y.variable_kind = 'integer'
    mdl = Model('sub')
    mdl.add_reactions(area_dict.values())
    # Global constraint: sum_a <= Area
    cons_area = Constraint('total_area')
    cons_area._bound = total_area
    cons_area._constraint_sense = 'L'
    mdl.add_metabolites(cons_area)
    for y in area_dict.values():
        y.add_metabolites({cons_area:1.}, combine=False)

    # MAX sell price
    x_excess = Variable('sell_excess_beet', lower_bound=0., upper_bound=UB)
    x_excess.objective_coefficient = -prob_dict[scen]*excess_sell_price    
    mdl.add_reaction(x_excess)
    
    for c in crops:
        ### Scenario variables x: tons sold, bought, excess sold per scenario
        x_sell = Variable('sell_%s'%(c), lower_bound=0., upper_bound=UB)
        x_buy  =  Variable('buy_%s'%(c), lower_bound=0., upper_bound=UB)
        mdl.add_reactions([x_sell,x_buy])        
        x_sell.objective_coefficient = -prob_dict[scen]*sellprice_dict[c]
        x_buy.objective_coefficient = prob_dict[scen]*buyprice_dict[c]
        
        ### Scenario constraint: Y_ck*ac - xs_ck + xb_ck >= Req_c        
        cons = Constraint('required_%s'%c)
        cons._bound = req_dict[c]
        cons._constraint_sense = 'G'        
        mdl.add_metabolites(cons)
        area = area_dict[c]
        area.add_metabolites({cons:yield_dict[scen][c]})
        x_sell.add_metabolites({cons:-1.})
        x_buy.add_metabolites({cons:1.})
        
        ### Beet quota
        if c=='beet':
            x_sell.upper_bound = beet_quota
            # xs_beet,k + xexcess_k <= Yield_beet,k * area_beet
            cons_beet = Constraint('beet_balance')
            cons_beet._bound = 0.
            cons_beet._constraint_sense = 'L'
            x_sell.add_metabolites({cons_beet:1.})
            x_excess.add_metabolites({cons_beet:1.})
            area_dict[c].add_metabolites({cons_beet:-yield_dict[scen][c]})
    
    sub = LagrangeSubmodel(mdl, scen)
    sub_dict[scen] = sub

In [6]:
master = LagrangeMaster(mdl)
master._INF = 1e7
master.add_submodels(sub_dict)
master._z.LB = -master._INF
master._z.UB = master._INF
master.model.update()

In [7]:
obj = master.model.getObjective()
obj

<gurobi.QuadExpr: z + [ -0.5 u_00 ^ 2 + -0.5 u_01 ^ 2 + -0.5 u_02 ^ 2 + -0.5 u_10 ^ 2 + -0.5 u_11 ^ 2 + -0.5 u_12 ^ 2 ]>

In [8]:
for v in master.model.getVars():
    print(v.VarName, v.LB, v.UB)

('z', -10000000.0, 10000000.0)
('tk_below', -10000000.0, 10000000.0)
('tk_average', -10000000.0, 10000000.0)
('tk_above', -10000000.0, 10000000.0)
('u_00', -10000000.0, 10000000.0)
('u_01', -10000000.0, 10000000.0)
('u_02', -10000000.0, 10000000.0)
('u_10', -10000000.0, 10000000.0)
('u_11', -10000000.0, 10000000.0)
('u_12', -10000000.0, 10000000.0)


In [9]:
for sub_ind,sub in iteritems(sub_dict):
    print('H_%s'%sub_ind)
    print(sub._H.todense())

H_below
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
H_average
[[-1.  0.  0.]
 [ 0. -1.  0.]
 [ 0.  0. -1.]
 [ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0.  1.]]
H_above
[[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [-1.  0.  0.]
 [ 0. -1.  0.]
 [ 0.  0. -1.]]


In [10]:
cc = master.model.getConstrs()[0]
print(master.model.getRow(cc), cc.Sense, cc.RHS)

(<gurobi.LinExpr: z + -1.0 tk_below + -1.0 tk_average + -1.0 tk_above>, '<', 0.0)


In [11]:
master.model.getObjective()

<gurobi.QuadExpr: z + [ -0.5 u_00 ^ 2 + -0.5 u_01 ^ 2 + -0.5 u_02 ^ 2 + -0.5 u_10 ^ 2 + -0.5 u_11 ^ 2 + -0.5 u_12 ^ 2 ]>

In [12]:
cz = master.model.getConstrByName('z_cut')
print(master.model.getRow(cz), cz.Sense, cz.RHS)

(<gurobi.LinExpr: z + -1.0 tk_below + -1.0 tk_average + -1.0 tk_above>, '<', 0.0)


In [13]:
from dynamicme.callback_gurobi import cb_benders_multi

In [14]:
# Need to scale variables? Since values are huge?

In [15]:
master.model.Params.Presolve = 0
# master.model.Params.ScaleFlag = 2
# master.model.Params.NumericFocus = 3
master.model.Params.BarHomogeneous = 1

# master.model.Params.Method = 0  # primal simplex
# master.model.Params.Method = 1  # dual simplex
master.model.Params.Method = 2  # barrier

master.model.Params.OutputFlag=0
master.max_iter = 100
master.print_iter=1
master.verbosity = 0

master.delta_mult = 0.01 #0.5   #0.1
master.delta_min  = 1e-10 #0

if RELAX_LP:
    uopt = master.solve_relaxed()
else:    
    uopt = master.optimize() 

        Iter          UB          LB    Best Bnd         gap   relgap(%)       delta     time(s)
           0       1e+07  -1.154e+05  -1.154e+05   1.012e+07       101.2           1    0.014098
           1   1.453e+06  -7.428e+06  -1.154e+05   1.569e+06       107.9        0.01    0.023428
           2       1e+07  -1.513e+08  -1.154e+05   1.012e+07       101.2      0.0001    0.032299
           3  -1.053e+05  -1.396e+05  -1.154e+05   1.012e+04       9.617       1e-06    0.039208
           4  -1.067e+05  -1.141e+05  -1.141e+05        7453       6.986       1e-08    0.045264
           5  -1.077e+05  -1.139e+05  -1.139e+05        6156       5.716       1e-10    0.048712
           6  -1.084e+05  -1.084e+05  -1.084e+05       44.95     0.04147       1e-10    0.051306
           7  -1.084e+05  -1.084e+05  -1.084e+05   6.563e-07   6.055e-10       1e-10    0.053849
relgap (6.05491e-12) <= gaptol (0.0001). Finished.


In [16]:
print(master._z.X)
master.model.getObjective()

-108390.000001


<gurobi.QuadExpr: z + [ -5e-11 u_00 ^ 2 + -5e-11 u_01 ^ 2 + -5e-11 u_02 ^ 2 + -5e-11 u_10 ^ 2 + -5e-11 u_11 ^ 2 + -5e-11 u_12 ^ 2 ]>

In [17]:
master.model

<gurobi.Model Continuous instance master: 22 constrs, 10 vars, Parameter changes: Method=2, BarHomogeneous=1, LogFile=gurobi.log, Presolve=0, OutputFlag=0>

In [18]:
cons = master.model.getConstrs()[1]
print(master.model.getRow(cons), cons.Sense, cons.RHS)
cons = master.model.getConstrs()[2]
print(master.model.getRow(cons), cons.Sense, cons.RHS)
cons = master.model.getConstrs()[3]
print(master.model.getRow(cons), cons.Sense, cons.RHS)

(<gurobi.LinExpr: tk_below + -375.0 u_00 + -25.0 u_01 + -100.0 u_02>, '<', -19983.333333333336)
(<gurobi.LinExpr: tk_average + 300.0 u_00 + 80.0 u_01 + 120.0 u_02 + -300.0 u_10 + -80.0 u_11 + -120.0 u_12>, '<', -39533.33333333334)
(<gurobi.LinExpr: tk_above + 250.0 u_10 + 67.0 u_11 + 183.0 u_12>, '<', -55883.33333333333)


In [19]:
# cons = master.model.getConstrs()[4]
# print(master.model.getRow(cons), cons.Sense, cons.RHS)
# cons = master.model.getConstrs()[5]
# print(master.model.getRow(cons), cons.Sense, cons.RHS)
# cons = master.model.getConstrs()[6]
# print(master.model.getRow(cons), cons.Sense, cons.RHS)

In [20]:
# cons = master.model.getConstrs()[7]
# print(master.model.getRow(cons), cons.Sense, cons.RHS)
# cons = master.model.getConstrs()[8]
# print(master.model.getRow(cons), cons.Sense, cons.RHS)
# cons = master.model.getConstrs()[9]
# print(master.model.getRow(cons), cons.Sense, cons.RHS)

## Check answer: should be 
- {'wheat':170, 'corn':80, 'beet':250}
- objval = (-)108390

In [21]:
master._z

<gurobi.Var z (value -108390.000001)>

In [22]:
x_dict = {}
for scen,sub in iteritems(master.sub_dict):
    #sub.update_obj(yopt)
    #sub.model.optimize()
    #x_dict[scen] = {v.VarName:v.X for v in sub.model.getVars()}
    x_dict[scen] = sub.x_dict

In [23]:
x_dict

{'above': {'area_beet': 250.0,
  'area_corn': 183.0,
  'area_wheat': 67.0,
  'buy_beet': 0.0,
  'buy_corn': 0.0,
  'buy_wheat': 0.0,
  'sell_beet': 6000.0,
  'sell_corn': 418.8000000000005,
  'sell_excess_beet': 0.0,
  'sell_wheat': 0.9999999999999858},
 'average': {'area_beet': 0.0,
  'area_corn': 80.0,
  'area_wheat': 420.0,
  'buy_beet': 0.0,
  'buy_corn': 0.0,
  'buy_wheat': 0.0,
  'sell_beet': 0.0,
  'sell_corn': 0.0,
  'sell_excess_beet': 0.0,
  'sell_wheat': 850.0},
 'below': {'area_beet': 0.0,
  'area_corn': 0.0,
  'area_wheat': 500.0,
  'buy_beet': 0.0,
  'buy_corn': 240.0,
  'buy_wheat': 0.0,
  'sell_beet': 0.0,
  'sell_corn': 0.0,
  'sell_excess_beet': 0.0,
  'sell_wheat': 800.0}}

In [24]:
Hys = []
for sub_ind,sub in iteritems(sub_dict):
    yk = np.array([sub.x_dict[x.VarName] for x in sub._ys])
    Hyk = sub._H*yk
    Hys.append(Hyk)    
    print('Scenario: %s'%sub_ind)
    print('yk = %s'%[(k,v) for k,v in iteritems(sub.x_dict)])
    print('Hk*yk =%s' % Hyk)
    
print("\nsum_k Hk*yk=")
print(sum(Hys))
print("\n u = %s" % master.uopt)

Scenario: below
yk = [('buy_wheat', 0.0), ('area_beet', 0.0), ('area_corn', 0.0), ('sell_wheat', 800.0), ('buy_corn', 240.0), ('sell_excess_beet', 0.0), ('sell_corn', 0.0), ('sell_beet', 0.0), ('buy_beet', 0.0), ('area_wheat', 500.0)]
Hk*yk =[  0.   0. 500.   0.   0.   0.]
Scenario: average
yk = [('buy_wheat', 0.0), ('area_beet', 0.0), ('area_corn', 80.0), ('sell_wheat', 850.0), ('buy_corn', 0.0), ('sell_excess_beet', 0.0), ('sell_corn', 0.0), ('sell_beet', 0.0), ('buy_beet', 0.0), ('area_wheat', 420.0)]
Hk*yk =[   0.  -80. -420.    0.   80.  420.]
Scenario: above
yk = [('buy_wheat', 0.0), ('area_beet', 250.0), ('area_corn', 183.0), ('sell_wheat', 0.9999999999999858), ('buy_corn', 0.0), ('sell_excess_beet', 0.0), ('sell_corn', 418.8000000000005), ('sell_beet', 6000.0), ('buy_beet', 0.0), ('area_wheat', 67.0)]
Hk*yk =[   0.    0.    0. -250. -183.  -67.]

sum_k Hk*yk=
[   0.  -80.   80. -250. -103.  353.]

 u = [ -0.4126032  -14.4126032  -42.4126032   46.69835385 -40.30164615
 -56.96831

In [25]:
sum(master.uopt * sum(Hys))

-29873.33333489485

## Check that all constraints satisfied

In [26]:
# Total area
for sub_ind,sub in iteritems(sub_dict):
    yopt = [v.X for v in sub._ys]
    print('Scenario=%s. Total area constraint:'%sub_ind, sum(yopt) <= total_area)

('Scenario=below. Total area constraint:', True)
('Scenario=average. Total area constraint:', True)
('Scenario=above. Total area constraint:', True)


In [27]:
# Cattle feed crop requirement
print("Cattle feed crop requirement met?\n%s"%('-'*40))
for scen,sub in iteritems(sub_dict):
    y_dict = {v.VarName:v.X for v in sub._ys}
    for c,req in iteritems(req_dict):
        area = y_dict['area_%s'%c]
        xsell = x_dict[scen]['sell_%s'%(c)]
        xbuy = x_dict[scen]['buy_%s'%(c)]
        lhs = yield_dict[scen][c]*area - xsell + xbuy
        rhs = req
        print('%s, %s: %g >= %g, %s'%(scen,c,lhs,rhs,lhs>=rhs))

Cattle feed crop requirement met?
----------------------------------------
below, beet: 0 >= 0, True
below, corn: 240 >= 240, True
below, wheat: 200 >= 200, True
average, beet: 0 >= 0, True
average, corn: 240 >= 240, True
average, wheat: 200 >= 200, True
above, beet: 0 >= 0, True
above, corn: 240 >= 240, False
above, wheat: 200 >= 200, True
